In [239]:
import pandas as pd
path = 'Electric_Vehicle_Population_Data.csv'
data = pd.read_csv(path)
data.head(2)

,VIN (1-10),County,City,State,Postal Code,Model Year,Make,Model,Electric Vehicle Type,Clean Alternative Fuel Vehicle (CAFV) Eligibility,Electric Range,Base MSRP,Legislative District,DOL Vehicle ID,Vehicle Location,Electric Utility,2020 Census Tract
0,5YJXCAE26J,Yakima,Yakima,WA,98908.0,2018,TESLA,MODEL X,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,238,0,14.0,141151601,POINT (-120.56916 46.58514),PACIFICORP,5.307700e+10
1,JHMZC5F37M,Kitsap,Poulsbo,WA,98370.0,2021,HONDA,CLARITY,Plug-in Hybrid Electric Vehicle (PHEV),Clean Alternative Fuel Vehicle Eligible,47,0,23.0,171566447,POINT (-122.64681 47.73689),PUGET SOUND ENERGY INC,5.303509e+10


In [240]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130443 entries, 0 to 130442
Data columns (total 17 columns):
 #   Column                                             Non-Null Count   Dtype  
---  ------                                             --------------   -----  
 0   VIN (1-10)                                         130443 non-null  object 
 1   County                                             130440 non-null  object 
 2   City                                               130440 non-null  object 
 3   State                                              130443 non-null  object 
 4   Postal Code                                        130440 non-null  float64
 5   Model Year                                         130443 non-null  int64  
 6   Make                                               130443 non-null  object 
 7   Model                                              130221 non-null  object 
 8   Electric Vehicle Type                              130443 non-null  object

In [241]:
data.columns.values

array(['VIN (1-10)', 'County', 'City', 'State', 'Postal Code',
       'Model Year', 'Make', 'Model', 'Electric Vehicle Type',
       'Clean Alternative Fuel Vehicle (CAFV) Eligibility',
       'Electric Range', 'Base MSRP', 'Legislative District',
       'DOL Vehicle ID', 'Vehicle Location', 'Electric Utility',
       '2020 Census Tract'], dtype=object)

In [242]:
#Buat dataset baru dengan kolom-kolom yang diperlukan.
dropped_col = [
       'VIN (1-10)', 'State', 'Postal Code', 'Base MSRP', 
       'Legislative District', 'DOL Vehicle ID', 'Vehicle Location','2020 Census Tract']
df = data.drop(columns=dropped_col)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130443 entries, 0 to 130442
Data columns (total 9 columns):
 #   Column                                             Non-Null Count   Dtype 
---  ------                                             --------------   ----- 
 0   County                                             130440 non-null  object
 1   City                                               130440 non-null  object
 2   Model Year                                         130443 non-null  int64 
 3   Make                                               130443 non-null  object
 4   Model                                              130221 non-null  object
 5   Electric Vehicle Type                              130443 non-null  object
 6   Clean Alternative Fuel Vehicle (CAFV) Eligibility  130443 non-null  object
 7   Electric Range                                     130443 non-null  int64 
 8   Electric Utility                                   130440 non-null  object
dtypes: in

In [243]:
#Mengganti beberapa nama kolom dengan nama yang lebih singkat.
df.rename(columns={
    'Make': 'Maker', 
    'Electric Vehicle Type': 'EV Type', 
    'Clean Alternative Fuel Vehicle (CAFV) Eligibility': 'CAFV Eligibility'}, inplace=True)
df.columns.values

array(['County', 'City', 'Model Year', 'Maker', 'Model', 'EV Type',
       'CAFV Eligibility', 'Electric Range', 'Electric Utility'],
      dtype=object)

In [244]:
check_col = df[['EV Type', 'CAFV Eligibility']]
for col in check_col.columns:
    unique_values = df[col].unique()
    print('{}: {}'.format(col, unique_values))

EV Type: ['Battery Electric Vehicle (BEV)' 'Plug-in Hybrid Electric Vehicle (PHEV)']
CAFV Eligibility: ['Clean Alternative Fuel Vehicle Eligible'
 'Not eligible due to low battery range'
 'Eligibility unknown as battery range has not been researched']


In [245]:
#Mengganti nilai-nilai string pada kolom EV type, CAFV Eligibility, dan Electric Utility.
df['EV Type'].replace({
    'Battery Electric Vehicle (BEV)': 'BEV', 
    'Plug-in Hybrid Electric Vehicle (PHEV)': 'PHEV'}, inplace=True)
df['CAFV Eligibility'].replace({
    'Clean Alternative Fuel Vehicle Eligible': 'Eligible', 
    'Not eligible due to low battery range': 'Not eligible',
    'Eligibility unknown as battery range has not been researched': 'Unknown'}, inplace=True)
df['Electric Utility'].replace({
    'BONNEVILLE POWER ADMINISTRATION||AVISTA CORP||BIG BEND ELECTRIC COOP, INC': 'AVISTA CORP',
    'BONNEVILLE POWER ADMINISTRATION||AVISTA CORP||INLAND POWER & LIGHT COMPANY': 'AVISTA CORP',
    'BONNEVILLE POWER ADMINISTRATION||AVISTA CORP||PUD NO 1 OF ASOTIN COUNTY': 'AVISTA CORP',
    'PUGET SOUND ENERGY INC': 'PSE INC'}, inplace=True)
df.head()

,County,City,Model Year,Maker,Model,EV Type,CAFV Eligibility,Electric Range,Electric Utility
0,Yakima,Yakima,2018,TESLA,MODEL X,BEV,Eligible,238,PACIFICORP
1,Kitsap,Poulsbo,2021,HONDA,CLARITY,PHEV,Eligible,47,PSE INC
2,King,Seattle,2019,TESLA,MODEL 3,BEV,Eligible,220,CITY OF SEATTLE - (WA)|CITY OF TACOMA - (WA)
3,King,Seattle,2013,NISSAN,LEAF,BEV,Eligible,75,CITY OF SEATTLE - (WA)|CITY OF TACOMA - (WA)
4,Thurston,Lacey,2017,TESLA,MODEL S,BEV,Eligible,210,PSE INC


In [246]:
#Filter dataset berdasarkan kolom "Model Year" dan "Electric Utility".
years = [2016, 2017, 2018, 2019, 2020]
sort_cols = ['Model Year', 'Maker', 'Model'] 
utilities = ['AVISTA CORP', 'PACIFICORP', 'PSE INC']
df = df.loc[df['Model Year'].isin(years) & df['Electric Utility'].isin(utilities)].sort_values(sort_cols).reset_index(drop=True)
df.head(10)

,County,City,Model Year,Maker,Model,EV Type,CAFV Eligibility,Electric Range,Electric Utility
0,Snohomish,Bothell,2016,AUDI,A3,PHEV,Not eligible,16,PSE INC
1,Kitsap,Bainbridge Island,2016,AUDI,A3,PHEV,Not eligible,16,PSE INC
2,Kitsap,Bremerton,2016,AUDI,A3,PHEV,Not eligible,16,PSE INC
3,Kitsap,Bremerton,2016,AUDI,A3,PHEV,Not eligible,16,PSE INC
4,Kitsap,Bainbridge Island,2016,AUDI,A3,PHEV,Not eligible,16,PSE INC
5,Thurston,Olympia,2016,AUDI,A3,PHEV,Not eligible,16,PSE INC
6,Thurston,Rochester,2016,AUDI,A3,PHEV,Not eligible,16,PSE INC
7,Kitsap,Bremerton,2016,AUDI,A3,PHEV,Not eligible,16,PSE INC
8,Snohomish,Bothell,2016,AUDI,A3,PHEV,Not eligible,16,PSE INC
9,Snohomish,Bothell,2016,AUDI,A3,PHEV,Not eligible,16,PSE INC


In [247]:
df.shape

(11045, 9)

In [248]:
df.isnull().values.any()

False

In [249]:
df.duplicated().any()

True

In [250]:
df = df.drop_duplicates(keep='first')
df.duplicated().any()

False

In [251]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3174 entries, 0 to 11044
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   County            3174 non-null   object
 1   City              3174 non-null   object
 2   Model Year        3174 non-null   int64 
 3   Maker             3174 non-null   object
 4   Model             3174 non-null   object
 5   EV Type           3174 non-null   object
 6   CAFV Eligibility  3174 non-null   object
 7   Electric Range    3174 non-null   int64 
 8   Electric Utility  3174 non-null   object
dtypes: int64(2), object(7)
memory usage: 248.0+ KB


In [252]:
df.to_csv('EV_part1.csv', index=False)